In [12]:
using LinearAlgebra, SparseArrays, Arpack

In [16]:
# The solveLinearSystem() function is general for any A that has orthogonal eigenvectors.
# The complete set of eigenvectors comprise N^2 (where N=nx*ny*nz) values so this code
# is only useful for small debugging exercises.
function solveLinearSystem(A,f)
 # Solve Aϕ=f
 tol=1.e-12
    
 # Use eigen (from Julia package LinearAlgebra) - does not support sparse matrix format
 # E=eigen(A);
 # L=E.values;
 # V=E.vectors;
    
 # Alternate using eig from ARPACK (from Julia package Arpack) - works with sparse matrix format
 N=size(A)[1];
 Le=fill(0.,N)
 Ve=spzeros(N,N)
 lAR,vAR=eigs(A,nev=N,which=:LM,tol=0.0);
 neAR=size(lAR)[1]
 Le[1:neAR]=lAR;
 Ve[:,1:neAR]=vAR;
 L=Le;
 V=Ve;
 # End alternate using ARPACK
    
 # display(Matrix(V))
    
 # Get amplitudes, F, of eigenvectors that give f
 F=V'*f
 rL=map(x -> if (abs(x)>tol) 1.0/x;  else 0. ; end , L);
 # Get amplitudes, Φ, of eigenvectors that give ϕ
 Φ=F.*rL
 # Solve for ϕ given Φ
 ϕ=V*Φ
 # println("A ",A)
 println("A*ϕ ",A*ϕ)
 println("  f ",f)
 err=A*ϕ-f;
 println("err ",err)
 println("sum(err) ",sum(err))
 println("   ϕ",ϕ)
 return ϕ
end

solveLinearSystem (generic function with 1 method)

In [17]:
# The solve_poisson_3d_mbc_eig() function forms an A operator with cyclic boundary conditions in X and Y
# and Neumann is Z. The function is configured to have the same arguments as the FFT solver solve_poisson_3d_mbc()
function solve_poisson_3d_mbc_eig(f, Lx, Ly, Lz)
    # Solve Aϕ=f using slow and expensive eigenvector and eigenvalue
    # factorization. 
    nx, ny, nz = size(f)
    
    # == Form sparse A matrix from diagonals ==
    N=nx*ny*nz;
    dx=Lx/nx;dy=Ly/ny;dz=Lz/nz;
    rdx2=1/dx^2;rdy2=1/dy^2;rdz2=1/dz^2;
    # A matrix element values. mdi - interior main diag, mdze - z edge main diagonal, odx - x off diagonal, ody - y off diagonal, odz - z off diagonal
    mdi=-2*rdx2-2*rdy2-2*rdz2;
    mdze=-2*rdx2-2*rdy2-1*rdz2;
    odx=1*rdx2;
    ody=1*rdy2;
    odz=1*rdz2;
    
    md=vcat( fill(mdze,nx*ny),
         fill(mdi, N-2*nx*ny),
         fill(mdze,nx*ny)
       )
    A=spdiagm( -(N-1) =>fill(odx,1),
           -(N-nx)=>fill(ody,nx),
           -nx*ny =>fill(odz,(nz-1)*nx*ny),
           -nx    =>fill(ody,N-nx),
           -1     =>fill(odx,N-1),
           0      =>md,
           +1     =>fill(odx,N-1),
           +nx    =>fill(ody,N-nx),
           +nx*ny =>fill(odz,N-nx*ny),
           +(N-nx)=>fill(ody,nx),
           +(N-1) =>fill(odx,1)
         )
    
    # Create vector view of f
    fv=reshape(f,(N));

    # Normalize/regularize
    # Remove mean from f
    mnfv=sum(fv)./size(fv)
    fv=fv.-mnfv;
    # Scale elements of A and f 
    ma=reduce(max,(abs.(A)));
    A=A./ma;
    fv=fv./ma;
    mxfv=reduce(max,(abs.(fv)));
    fv=fv./mxfv;
    
    ϕ=solveLinearSystem(A,fv)
    # Unnormalize
    ϕ=ϕ.*mxfv;
    
end

solve_poisson_3d_mbc_eig (generic function with 1 method)

In [23]:
# The following illustrates code for using these functions.
@show Lx = 100
@show Ly = 200
@show Lz = 10
f = rand(7, 9, 5);

ϕ = solve_poisson_3d_mbc_eig(f, Lx, Ly, Lz)

Lx = 100 = 100
Ly = 200 = 200
Lz = 10 = 10


┌ Warning: Adjusting nev from 315 to 314
└ @ Arpack C:\Users\Ali\.julia\packages\Arpack\UiiMc\src\Arpack.jl:99


A*ϕ [0.852117, 0.254606, 0.648493, -0.703529, -0.102985, -0.907696, -0.5276, -0.84898, -0.196228, 0.0301836, -0.192591, 0.247187, -0.95072, -0.370043, 0.338371, 0.114515, 0.910657, -0.840043, -0.968624, 0.84665, 0.571314, -0.569702, 0.0219022, 0.864527, 0.00964873, 0.414233, -0.782697, -0.318611, 0.691659, 0.427753, 0.303885, 0.18076, 0.0602945, 0.0414831, 0.328133, 0.42098, -0.616012, -0.345445, -0.132687, 0.65877, 0.165164, -0.243532, 0.607671, -0.533482, -0.542928, -0.274663, 0.0520339, 0.180247, -0.38902, -0.470824, 0.546906, -0.797912, 0.416057, 0.792461, -0.936668, -0.334652, -0.235322, -0.239878, -0.166031, 0.595179, -0.846375, -0.654173, -0.0360288, 0.320356, 0.18802, 0.513348, -0.582805, 0.699748, -0.301995, -0.961064, 0.441978, 0.777152, -0.536236, -0.929076, -0.794685, 0.744349, 0.842977, -0.0829364, -0.457233, 0.273299, -0.566241, -0.221171, 0.158692, 0.822054, -0.88498, 0.853884, 0.0631232, 0.57325, -0.198596, 0.318804, 0.598352, -0.968902, -0.44365, -0.931524, 0.738888, -

err [5.81757e-14, -1.45106e-13, 4.21885e-15, -4.996e-15, 3.36953e-14, -7.03881e-14, 4.90719e-14, 1.19349e-13, 4.19387e-14, -5.29715e-14, 4.25215e-14, 5.4956e-14, 2.55351e-14, 4.05231e-15, 7.10543e-15, -1.25081e-13, -5.32907e-15, -6.99441e-14, 4.52971e-14, -2.02061e-14, 3.35287e-14, -5.14033e-14, 3.16761e-15, -5.71765e-14, 2.13891e-14, -7.43849e-15, 3.57492e-14, 2.7367e-14, 8.26006e-14, 8.83738e-14, 6.12843e-14, 9.46465e-14, 5.32699e-14, 5.84741e-14, 6.35048e-14, 5.86753e-14, -1.07692e-14, 8.85403e-14, 8.15181e-14, 3.9968e-14, -1.0078e-13, 5.32907e-15, 7.63833e-14, 1.19904e-14, 3.09752e-14, -6.08957e-14, 2.07195e-14, 5.68989e-14, 3.55826e-14, -2.87548e-14, 2.32037e-14, -2.22045e-16, -3.46945e-14, 3.64153e-14, 1.72085e-14, 7.99361e-15, -3.16691e-14, -3.9857e-14, 5.82867e-15, 2.4647e-14, 7.82707e-14, -5.62883e-14, -6.40876e-14, -9.10383e-15, 1.29036e-13, -1.06248e-13, 1.0103e-14, 3.58602e-14, 1.01752e-13, -2.36478e-14, -1.08968e-13, -3.70814e-14, 4.27436e-14, -7.07212e-14, -9.74776e-14, -

315-element Array{Float64,1}:
 19.588710219329187 
 11.767638786158095 
  1.3394900182637857
 22.929384519312332 
 29.90561690318779  
 33.23901302458847  
 36.63679710953569  
 20.49301626354344  
 14.128330807371961 
 15.093809164823814 
 12.752164981053129 
  9.411121173757781 
  7.23564210373399  
  ⋮                 
 14.137957181834043 
  8.378729912993949 
 19.292181037420843 
 46.521965359066044 
 29.40860997930564  
 29.158932011967728 
  6.683304434629206 
 12.934125897742478 
 12.36231460131163  
 31.141269895802285 
 50.26002416838233  
 37.38753420252298  

In [25]:
function dd(n)
    d = diagm(0 => -2*ones(n), 1 => ones(n-1), -1 => ones(n-1))
    d[1, end] = 1; d[end, 1] = 1;
    d
end

dd (generic function with 1 method)

In [28]:
A = dd(5)

5×5 Array{Float64,2}:
 -2.0   1.0   0.0   0.0   1.0
  1.0  -2.0   1.0   0.0   0.0
  0.0   1.0  -2.0   1.0   0.0
  0.0   0.0   1.0  -2.0   1.0
  1.0   0.0   0.0   1.0  -2.0

In [29]:
f = rand(5)

5-element Array{Float64,1}:
 0.5903920852627205
 0.7415230120821314
 0.832574406590886 
 0.5811216193778135
 0.7545240042928185

In [32]:
x = solveLinearSystem(A, f)

A*ϕ [-0.109635, 0.041496, 0.132547, -0.118905, 0.054497]
  f [0.590392, 0.741523, 0.832574, 0.581122, 0.754524]
err [-0.700027, -0.700027, -0.700027, -0.700027, -0.700027]
sum(err) -3.5001351276063697
   ϕ[0.0465824, -0.0294801, -0.0640465, 0.0339344, 0.0130099]


┌ Warning: Adjusting nev from 5 to 4
└ @ Arpack C:\Users\Ali\.julia\packages\Arpack\UiiMc\src\Arpack.jl:99


5-element Array{Float64,1}:
  0.04658237108865165 
 -0.029480080098726172
 -0.06404654472524655 
  0.033934371717845   
  0.013009882017476104